<a href="https://colab.research.google.com/github/GitEmmSt/Medical_Imaging/blob/main/Practical_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Medical Imaging/practical_session_2

/content/drive/MyDrive/Medical Imaging/practical_session_2


# <center>Medical Imaging</center><br><center>Practical session 2: Monte Carlo simulation (19/10/2021)</center>
***
*Charlotte Thyssen, Jens Maebe, Stefaan Vandenberghe* <br>
*MEDISIP, Ghent University* <br>

# General
In total, there will be 4 practical sessions. For every session you should hand in a report **in groups of 2**. These reports will count for 5 out of 20 points for the exam. The topics of the sessions:

1. Image reconstruction in tomography, CT (25%)	

2. Monte Carlo simulations, SPECT (25%)	

3. Image processing part I (25%)

4. Image processing part II (25%)

***
Report (this notebook) due 2 weeks after session


- **Concise** and **to the point** answers to the questions

- Show that you understood what was going on and what the purpose of the exercise was

- Illustrate with figures (do not forget labels, legend, colorbar etc.)

- Hand in your notebook (also add the HTML file, File -> Download as -> HTML) via UFORA (UGent) or Canvas (VUB)

***
We are here to help you: ask questions!
- During the sessions
- Via mail:
    - cathysse.thyssen@ugent.be
    - jens.maebe@ugent.be
    

**Strict deadline for this session: 1 November 2021 at 23:59**.

# Notebook
This practical session includes a number of pre-defined functions for you to make use of. **You do not need to change these in any way.** Make sure to check the expected input/output variables for these functions when you use them in your code. They are declared during their relevant sections. Also make sure you have `numpy` and `matplotlib` installed and run the cell below in order to import them.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Purpose

The purpose of this exercise session is to obtain insight in the different steps involved in the Monte Carlo (MC) simulation of a Single Photon Emission Computed Tomography (SPECT) scanner. We will use MC techniques to simulate a SPECT acquisition, instead of using analytical forward projections.

# 2. Background: the SPECT camera

A SPECT scanner measures projections of the distribution of a pharmacon, labeled with a radioactive isotope, in the body. By measuring projections along different angles, a sinogram is obtained that can be reconstructed to a 3D image (also see the previous practicum). Since the labeled pharmacon is subject to a certain metabolism in the patient, a 3D image of the physiology can be obtained with a SPECT camera. The technique is thus complementary to a CT scan, which images only anatomy.

As shown in Figure 1, three major components are involved in a SPECT acquisition. In order to obtain a measurement, we first need a **source** of photons, usually a patient injected with a radiotracer. Next we need a photon **detector** to measure the photons that leave the patient’s body. A third essential component of the gamma camera is the **collimator**. This is needed to sort out the desired direction. Many types of collimators exist, such as the parallel-hole collimator, the pinhole collimator, fan-beam collimators, etc, each with their own advantages and disadvantages. In the case of a parallel hole collimator, we sort out the direction perpendicular to the detector surface.

A simple parallel hole collimator would be made from a lead slab of material with parallel holes drilled through this plate. These holes can be hexagonal, square or round. The detector is a scintillator crystal which converts gamma photons to visible light. The light output can then be amplified into a detectable signal by photomultipliers and processed by a computer, which forms the sinogram.

<table>
    <tr>
        <td><img src="images/SPECT_parts.jpg" width="300"/><figcaption><center>(a)</center></figcaption></td>
        <td><img src="images/SPECT_scanner.jpg" width="300"/><figcaption><center>(b)</center></figcaption></td>
    </tr>
</table>

<center><i>Figure 1: (a) The different parts of a gamma camera. (b) A modern SPECT scanner with two heads.</i></center>

# 3. Monte Carlo simulations

Monte Carlo simulations are widely used in physics and mathematics when the analytical calculation of a result is too difficult or sometimes even impossible to compute. The method is named after the Monte Carlo Casino, because it relies on repeated **random sampling** or educated guesses to compute a numerical result.

The first random number we will use in our SPECT simulator will be used to pick a random point of photon emission, given a certain source density map.

## 3.1. Sampling of the source

The activity distribution in a patient can be represented by voxels with linear index ($j\in1···N$) with a certain value ($n_j$) representing the true activity in an arbitrary unit. The first step in our MC simulation will be to randomly select one voxel, taking into account its activity (a higher activity means more photons should originate from that location). The activity distribution can be considered as a probability density function (PDF): the probability of emission in one voxel will be proportional to its activity $n_j$.

In order to simulate these emissions, we need to sample the PDF. We do this by using the cumulative probability density (CPD), obtained by integrating the PDF. In the case of a discrete system, we have:

$$CPD(j) = \sum_{k=1}^{j}{n_k}$$

Next, we normalize the CPD to 1 by dividing by CPD(N). We can now select a random number $r_1$ between 0 and 1, and can relate this number to a voxel by scaling it back to the index using the CPD (see figure 2).

<table>
    <tr>
        <td><img src="images/vox_phantom.jpg" width="300"/><figcaption><center>(a)</center></figcaption></td>
        <td><img src="images/pdf.jpg" width="300"/><figcaption><center>(b)</center></figcaption></td>
        <td><img src="images/cpd.jpg" width="300"/><figcaption><center>(c)</center></figcaption></td>
    </tr>
</table>

*Figure 2: (a) Representation of the activity distribution by a voxelized source. (b) The activity represented as a probability density function (PDF). (c) The cumulative probability density (CPD) on which a voxel is chosen from a random number $r_1$.*

## <font color='blue'>Exercise 1: sample a uniform cylinder</font>

Read in the phantom "phantoms/cyl_25.img" in `np.float32` format using the `read_phantom` function provided below. The dimensions are 128 × 128 × 128 voxels. `show_phantom` can be used to visualize the phantom. Create and normalize the CPD from this phantom and then sample the source n times. **Because pure Python loops are slow, you should use `numpy` to sample n voxels at once (allowing for repetition) without the use of a for loop.** Use the sampled voxels to fill a new 128 × 128 × 128 image where each voxel value equals the amount of times this voxel was sampled.

Sample the source n = 100, 200, 300, ..., 9900, 10000 times and for each run, calculate the mean and standard deviation of the sampled image in order to make a plot of the mean as a function of the standard deviation. **Make sure to calculate the metrics only on the support (where the phantom is defined, i.e. different from zero).** Visualize the sampled image for different sample counts with `show_phantom` and compare to the original phantom. 

What relationship do you observe between the mean and the standard deviation and explain why?

### <font color='blue'>Please include in your notebook:</font>
- A few sampled images for different sample counts n (displayed with `show phantom`).
- A plot of mean (y-axis) versus standard deviation (x-axis).
- The mathematical relationship observed in this plot.
- An explanation for why you observe this relationship.

### Provided functions:

In [ ]:
def read_phantom(path, shape, dtype):
    with open(path, 'rb') as f:
        phantom = np.fromfile(f, dtype).reshape(shape)
    return phantom

`read_phantom` reads a voxelized source (also called a phantom)

**inputs:**
- `path`: relative or full path of the file (`str`)
- `shape`: dimensions of the phantom (`tuple`)
- `dtype`: datatype (`type`)

**output:**
- `phantom`: voxelized source (`np.ndarray`)

In [ ]:
def show_phantom(phantom):
    
    with plt.rc_context({'figure.figsize':(16,4)}):
        fig, axes = plt.subplots(ncols=3)
        ax0, ax1, ax2 = axes
        
        # x-projection
        im0 = ax0.imshow(np.sum(phantom, axis=0).T, origin='lower')
        ax0.set_title('x-projection')
        ax0.set_xlabel('iy')
        ax0.set_ylabel('iz')
        ax0.set_aspect('equal')
        fig.colorbar(im0, ax=ax0)
        
        # y-projection
        im1 = ax1.imshow(np.sum(phantom, axis=1).T, origin='lower')
        ax1.set_title('y-projection')
        ax1.set_xlabel('ix')
        ax1.set_ylabel('iz')
        ax1.set_aspect('equal')
        fig.colorbar(im1, ax=ax1)
        
        # z-projection
        im2 = ax2.imshow(np.sum(phantom, axis=2).T, origin='lower')
        ax2.set_title('z-projection')
        ax2.set_xlabel('ix')
        ax2.set_ylabel('iy')
        ax2.set_aspect('equal')
        fig.colorbar(im2, ax=ax2)
        
        plt.show()

`show_phantom` displays the phantom

**input:**
- `phantom`: voxelized source (`np.ndarray`)

### <font color='red'>Exercise 1: solution</font>

## 3.2. Simulating uniform emission

To sample the angle of emission uniformly over the complete solid angle ($4\pi$), we use the spherical
coordinate system as defined in figure 3. When the PDF of the azimuthal angle $\theta$ is chosen to be
uniform, the PDF of the polar angle $\phi$ will have to be weighted in order to obtain a uniform sampling
over $4\pi$ or, in other words, over the surface of the unit sphere.

<img src="images/SphericalCoordinates.jpg" width="300"/>

<center><i>Figure 3: Polar coordinate system with radial coordinate $r$, azimuthal angle $\theta$ and polar angle $\phi$.</i></center>

### Sampling of $\theta$

The PDF for $\theta$ is uniform and given by:

$$PDF(\theta) = \frac{1}{\theta_{max}}$$

The constant value $1/\theta_{max}$ is chosen so the CPD is normalized to 1, where $\theta_{max}=2\pi$. The Cumulative PDF (CPD) is then given by:

$$CPD(\theta) = \int_{0}^{\theta}{\frac{1}{\theta_{max}}d\theta'} = \frac{\theta}{\theta_{max}}$$

We choose the random number $r_2$ ($0 \leq r_2<1$) to be equal to the CPD:

$$r_2 = CPD(\theta) = \frac{\theta}{\theta_{max}}$$

so that $\theta$ can be derived from the random number $r_2$:

$$\theta = r_2\theta_{max}$$

### Sampling of $\phi$

If the PDF of $\phi$ would be uniform, the sampling of the surface of the unit sphere would be denser at the
poles while becoming coarser toward the equator. In order to have a uniform sampling over the unit sphere, the PDF of the
polar angle $\phi$ has to be chosen proportional to the circumference of the circle on the unit sphere defined
by $\phi$ (see figure 3).

$$PDF(\phi) = \text{circumference}(\phi)$$

Since we want the polar angle to be in between $-\phi_{max}/2$ and $\phi_{max}/2$, the normalization is calculated
by:

$$N_{PDF} = \int_{-\phi_{max}/2}^{\phi_{max}/2}PDF(\phi)d\phi$$

and the CPD then becomes:

$$CPD(\phi) = \frac{1}{N_{PDF}}\int_{-\phi_{max}/2}^{\phi}PDF(\phi')d\phi'$$

Using above formulas, we can show that the sampling of $\phi$ should be equal to:

$$\phi = \arcsin[(2r_3-1)\sin(\phi_{max}/2)]$$

with $r_3$ a uniformly chosen random number for which $0 \leq r_3<1$.

## <font color='blue'>Exercise 2: sampling of $\theta$ and $\phi$</font>

Simulate the sampling of $\theta$ and $\phi$ for $10^5$ emissions. Again make sure to use numpy to sample n angles at once rather than using a for loop. You can test your sampling with the function `test_spheresampling`.

First choose $\theta_{max}$ equal to $2\pi$ and $\phi_{max}$ equal to $\pi$. Then play around with $\theta_{max}$ and $\phi_{max}$ but do not choose values above $2\pi$ and $\pi$ respectively. Now try to sample sample $\phi$ 'naively', that is by using uniform sampling. What happens with the sphere sampling in this case?

### <font color='blue'>Please include in your notebook:</font>
- A plot of the sampling on the unit sphere for $\theta_{max}=2\pi$ and $\phi_{max}=\pi$.
- Some additional plots of the sampling for different values of $\theta_{max}$ and $\phi_{max}$.
- A plot of the sampling on the unit sphere where $\phi$ was sampled uniformly (again using $\theta_{max}=2\pi$ and $\phi_{max}=\pi$).
- A short explanation of what is happening in these cases.

Make use of `test_spheresampling` to generate these plots.

### Provided functions:

In [ ]:
def test_spheresampling(theta, phi):
    
    # make sure we are using numpy arrays and not lists
    theta = np.array(theta)
    phi = np.array(phi)
    
    # convert spherical coordinates to cartesian coordinates
    r = 1
    rcos = r * np.cos(phi)
    x = rcos * np.cos(theta)
    y = rcos * np.sin(theta)
    z = r * np.sin(phi)
    
    H, edges = np.histogramdd((x,y,z), bins=[np.linspace(-1.25,1.25,128)]*3)
    
    with plt.rc_context({'figure.figsize':(16,4)}):
        fig, axes = plt.subplots(ncols=3)
        ax0, ax1, ax2 = axes
        
        # x-projection
        Y, Z = np.meshgrid(edges[1], edges[2])
        im0 = ax0.pcolormesh(Y, Z, np.sum(H, axis=0).T)
        ax0.set_title('x-projection')
        ax0.set_xlabel('y')
        ax0.set_ylabel('z')
        ax0.set_aspect('equal')
        fig.colorbar(im0, ax=ax0)
        
        # y-projection
        X, Z = np.meshgrid(edges[0], edges[2])
        im1 = ax1.pcolormesh(X, Z, np.sum(H, axis=1).T)
        ax1.set_title('y-projection')
        ax1.set_xlabel('x')
        ax1.set_ylabel('z')
        ax1.set_aspect('equal')
        fig.colorbar(im1, ax=ax1)
        
        # z-projection
        X, Y = np.meshgrid(edges[0], edges[1])
        im2 = ax2.pcolormesh(X, Y, np.sum(H, axis=2).T)
        ax2.set_title('z-projection')
        ax2.set_xlabel('x')
        ax2.set_ylabel('y')
        ax2.set_aspect('equal')
        fig.colorbar(im2, ax=ax2)
        
        plt.show()

`test_spheresampling` visualizes the sampling on the unit sphere

**inputs:**
- `theta`: array of sampled $\theta$ values (`np.ndarray`)
- `phi`: array of sampled $\phi$ values (`np.ndarray`)

### <font color='red'>Exercise 2: solution</font>

## 3.3 Sinogram acquisition

### 3.3.1 Calculating intersection with collimator and detector

Once the place of emission (x, y, z) and the emission angles $\theta$ and $\phi$ are sampled, we can calculate the
intersection with the collimator and/or detector. After we choose a random detector angle $\Theta$ (chosen uniformly around the z-axis), x and y become the rotated x’ and y’ and the angle $\theta$ becomes $\theta-\Theta$. From figure 4 we calculate:

$$ y_{det} = (radius-x') \tan(\theta-\Theta) + y' $$

$$ z_{det} = (radius-x') \frac{\tan(\phi)}{\cos(\theta-\Theta)} + z $$

Calculating the collimator coordinates is very similar. We only have to subtract the collimator height from the radius:

$$ y_{coll} = (radius-height-x') \tan(\theta-\Theta) + y' $$
$$ z_{coll} = (radius-height-x') \frac{\tan(\phi)}{\cos(\theta-\Theta)} + z $$

with:

$$ x' = x\cos(\Theta) + y\sin(\Theta) $$
$$ y' = y\cos(\Theta) - x\sin(\Theta) $$

<table>
    <tr>
        <td><img src="images/detector_y.jpg" width="400"/><figcaption><center>(a)</center></figcaption></td>
        <td><img src="images/detector_z.jpg" width="400"/><figcaption><center>(b)</center></figcaption></td>
    </tr>
</table>

<center><i>Figure 4: (a) The geometry to calculate y_det. (b) The geometry to calculate z_det.</i></center>

### 3.3.2. Check if the photon passed the collimator

If the photon intersects with both the detector and collimator, we should check whether the photon was able to fly through one of the collimator holes. This is done by dividing both the detector and collimator coordinates (respectively $(y_{det},z_{det})$ and $(y_{coll},z_{coll})$) by the hole size. If we round both divisions and they are equal, both $(y_{det},z_{det})$ and $(y_{coll},z_{coll})$ lie within the same collimator hole. This results in a photon that passed the collimator. These conditions are:

$$ \left\lfloor{\frac{y_{det}}{hole\_size}}\right\rfloor == \left\lfloor{\frac{y_{coll}}{hole\_size}}\right\rfloor $$

$$ \left\lfloor{\frac{z_{det}}{hole\_size}}\right\rfloor == \left\lfloor{\frac{z_{coll}}{hole\_size}}\right\rfloor $$

## <font color='blue'>Exercise 3: full Monte Carlo simulation</font>

Read in the 128 × 128 × 128 "phantoms/mcatnohdr.bin" image in `np.int16` format. Perform a SPECT Monte Carlo simulation of this phantom as explained troughout the notebook. You will have to reuse your code to generate the CPD, sample the source, sample $\theta$ and sample $\phi$. Additionally, you will have to sample the detector angle $\Theta$ uniformly from 0 to $2\pi$. Use the function `vox2pos` defined below to obtain the absolute coordinates (x, y, z) of the sampled voxels. Additional functions implementing the above conditions for photon detection are also provided.

Use $10^6$ emissions, $\theta_{max}=2\pi$, $\phi_{max}=\pi/20$, $\Theta_{max}=2\pi$, detector sizes (y and z) 128, detector radius 150, collimator height 25 and hole size 1. Finally, use the function `generate_sinogram` to convert the calculated/sampled values of $y_{det}$, $z_{det}$ and $\Theta$ into a 3D sinogram (with axes corresponding to $y_{det}$, $z_{det}$ and $\Theta$), **making sure to only include those events that obey the above conditions for photon detection.** Use 90 detection angles spread over the full $2\pi$ range for the sinogram.

Take a look at the sinogram of the center slice along the detector z-axis: `sinogram[:,64,:]`. As you will see, we do not have many detections. In order to obtain a decent sinogram with enough statistics, we would need a lot more counts. Since we do not have time to wait for a long simulation, simulate a point source as follows:

`phantom = np.zeros((128,128,128))` 

`phantom[96,96,64] = 1`

Redo the Monte Carlo simulation and count the number of detections (total over all slices, so the full 3D sinogram) and take a look at the aforementioned center slice of the sinogram.

Now change the collimator hole size to 9. What happens with the number of detections and the sinogram? Explain!

### <font color='blue'>Please include in your notebook:</font>
- A plot of both sinograms (center slice only) **of the point source** (for hole size 1 as well as for hole size 9).
- The total (over the full 3D sinogram) number of detections in both cases.
- Motivate why you observe this (think about sensitivity vs resolution).

### Provided functions:

In [ ]:
def vox2pos(voxels, shape=(128,128,128)):
    positions = tuple(ind - s/2 + np.random.rand(len(ind)) for ind, s in zip(voxels, shape))
    return positions

`vox2pos` finds the absolute locations `(x,y,z)` of voxels `(ix,iy,iz)` in 3D Cartesian coordinates. It also randomizes the point of emission uniformly inside the voxel itself.

**inputs:**
- `voxels`: tuple of lists of voxel indices `([ix0, ix1, ...], [iy0, iy1, ...], [iz0, iz1, ...])`
- `shape`: dimensions of the phantom (`tuple`)

**output:**
- `positions`: tuple of lists of emission coordinates `([x0, x1, ...], [y0, y1, ...], [z0, z1, ...])`

In [ ]:
def hit_detector(x, y, z, theta, phi, Theta, size_y=128, size_z=128, radius=150):
    x_rot = x*np.cos(Theta) + y*np.sin(Theta)
    y_rot = y*np.cos(Theta) - x*np.sin(Theta)
    y_det = (radius-x_rot) * np.tan(theta-Theta) + y_rot
    z_det = (radius-x_rot) * np.tan(phi) / np.cos(theta-Theta) + z
    det_flag = (-size_y/2 <= y_det) & (y_det <= size_y/2) & (-size_z/2 <= z_det) & (z_det <= size_z/2)
    return det_flag, y_det, z_det

`hit_detector` calculates the detector intersection coordinates and checks whether the point
lies within the boundaries of the detector. Inputs of type `np.ndarray` should consist of a numpy array containing the sampled values for n events.

**inputs:**
- `x`: x-coordinates of the emissions (`np.ndarray`)
- `y`: y-coordinates of the emissions (`np.ndarray`)
- `z`: z-coordinates of the emissions (`np.ndarray`)
- `theta`: emission angles $\theta$ (`np.ndarray`)
- `phi`: emission angles $\phi$ (`np.ndarray`)
- `Theta`: detector angles $\Theta$ (`np.ndarray`)
- `size_y`: detector size y (`float`)
- `size_z`: detector size z (`float`)
- `radius`: detector radius (`float`)

**outputs:**
- `det_flag`: array of booleans corresponding to hits that fall within the boundaries of the detector (`np.ndarray`)
- `y_det`: array of detector intersection coordinates $y_{det}$ (`np.ndarray`)
- `z_det`: array of detector intersection coordinates $z_{det}$ (`np.ndarray`)

In [ ]:
def hit_collimator(x, y, z, theta, phi, Theta, size_y=128, size_z=128, radius=150, height=25):
    x_rot = x*np.cos(Theta) + y*np.sin(Theta)
    y_rot = y*np.cos(Theta) - x*np.sin(Theta)
    y_coll = (radius-height-x_rot) * np.tan(theta-Theta) + y_rot
    z_coll = (radius-height-x_rot) * np.tan(phi) / np.cos(theta-Theta) + z
    coll_flag = (-size_y/2 <= y_coll) & (y_coll <= size_y/2) & (-size_z/2 <= z_coll) & (z_coll <= size_z/2)
    return coll_flag, y_coll, z_coll

`hit_collimator` calculates the collimator intersection coordinates and checks whether the point
lies within the boundaries of the collimator. Inputs of type `np.ndarray` should consist of a numpy array containing the sampled values for n events.

**inputs:**
- `x`: x-coordinates of the emissions (`np.ndarray`)
- `y`: y-coordinates of the emissions (`np.ndarray`)
- `z`: z-coordinates of the emissions (`np.ndarray`)
- `theta`: emission angles $\theta$ (`np.ndarray`)
- `phi`: emission angles $\phi$ (`np.ndarray`)
- `Theta`: detector angles $\Theta$ (`np.ndarray`)
- `size_y`: detector size y (`float`)
- `size_z`: detector size z (`float`)
- `radius`: detector radius (`float`)
- `height`: collimator height (`float`)

**outputs:**
- `det_flag`: array of booleans corresponding to hits that fall within the boundaries of the collimator (`np.ndarray`)
- `y_col`: array of collimator intersection coordinates $y_{col}$ (`np.ndarray`)
- `z_col`: array of collimator intersection coordinates $z_{col}$ (`np.ndarray`)

In [ ]:
def pass_collimator(y_det, z_det, y_coll, z_coll, hole_size):
    y_det_hole = np.floor(y_det/hole_size)
    z_det_hole = np.floor(z_det/hole_size)
    y_coll_hole = np.floor(y_coll/hole_size)
    z_coll_hole = np.floor(z_coll/hole_size)
    pass_flag = (y_coll_hole==y_det_hole) & (z_coll_hole==z_det_hole)
    return pass_flag

`pass_collimator` determines whether the photon passed the collimator or not

**inputs:**
- `y_det`: detector intersection coordinates $y_{det}$ (`np.ndarray`)
- `z_det`: detector intersection coordinates $z_{det}$ (`np.ndarray`)
- `y_coll`: collimator intersection coordinates $y_{coll}$ (`np.ndarray`)
- `z_coll`: collimator intersection coordinates $z_{coll}$ (`np.ndarray`)
- `hole_size`: hole size of the collimator (`float`)

**outputs:**
- `pass_flag`: array of booleans corresponding to hits that pass the collimator (`np.ndarray`)

In [ ]:
def generate_sinogram(y_det, z_det, Theta, size_y=128, size_z=128, num_angles=90):
    angle_step = 2*np.pi / num_angles
    iy_det = (y_det + size_y/2).astype(int)
    iz_det = (z_det + size_z/2).astype(int)
    angle_det = (Theta/angle_step).astype(int)
    sinogram = np.zeros((size_y,size_z,num_angles)) 
    for iy, iz, angle in zip(iy_det, iz_det, angle_det):
        sinogram[iy,iz,angle] += 1
    return sinogram

`generate_sinogram` generates the 3D sinogram of the detected events

**inputs:**
- `y_det`: array of detector intersection coordinates $y_{det}$ (`np.ndarray`)
- `z_det`: array of detector intersection coordinates $z_{det}$ (`np.ndarray`)
- `Theta`: array of detector angles $\Theta$ (`np.ndarray`)
- `size_y`: detector size y (`float`)
- `size_z`: detector size z (`float`)
- `angle_step`: angular step size of the sinogram in radians (`float`)

**outputs:**
- `sinogram`: the 3D sinogram (`np.ndarray`)

### <font color='red'>Exercise 3: solution</font>